<a href="https://colab.research.google.com/github/seriamedard/ASL_ResNet/blob/main/ASL_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocessing


In [39]:
"""
import os
os.environ["KAGGLE_CONFIG_DIR"] = '/content/drive/MyDrive/data_asl'
%cd /content/drive/MyDrive/data_asl
!ls
!kaggle datasets download -d grassknoted/asl-alphabet
!mkdir asl_recognition
!mv asl-alphabet.zip asl_recognition
%cd asl_recognition/
!unzip asl-alphabet.zip
import cv2
from google.colab.patches import cv2_imshow
img=cv2.imread('/content/drive/MyDrive/data_asl/asl_recognition/asl_alphabet_test/asl_alphabet_test/A_test.jpg')
cv2_imshow(img)
"""

'\nimport os\nos.environ["KAGGLE_CONFIG_DIR"] = \'/content/drive/MyDrive/data_asl\'\n%cd /content/drive/MyDrive/data_asl\n!ls\n!kaggle datasets download -d grassknoted/asl-alphabet\n!mkdir asl_recognition\n!mv asl-alphabet.zip asl_recognition\n%cd asl_recognition/\n!unzip asl-alphabet.zip\nimport cv2\nfrom google.colab.patches import cv2_imshow\nimg=cv2.imread(\'/content/drive/MyDrive/data_asl/asl_recognition/asl_alphabet_test/asl_alphabet_test/A_test.jpg\')\ncv2_imshow(img)\n'

In [60]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2


In [47]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [52]:
train_folder = '/content/drive/MyDrive/data_asl/asl_recognition/asl_alphabet_train/asl_alphabet_train'
all_data = []
for folder in os.listdir(train_folder):
    
    label_folder = os.path.join(train_folder, folder)
    onlyfiles = [{'label':folder,'path':os.path.join(label_folder, f)} for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))]
    #print(onlyfiles)
    all_data += onlyfiles
data_df = pd.DataFrame(all_data)


In [55]:
data_df.head

<bound method NDFrame.head of        label                                               path
0          A  /content/drive/MyDrive/data_asl/asl_recognitio...
1          A  /content/drive/MyDrive/data_asl/asl_recognitio...
2          A  /content/drive/MyDrive/data_asl/asl_recognitio...
3          A  /content/drive/MyDrive/data_asl/asl_recognitio...
4          A  /content/drive/MyDrive/data_asl/asl_recognitio...
...      ...                                                ...
86995  space  /content/drive/MyDrive/data_asl/asl_recognitio...
86996  space  /content/drive/MyDrive/data_asl/asl_recognitio...
86997  space  /content/drive/MyDrive/data_asl/asl_recognitio...
86998  space  /content/drive/MyDrive/data_asl/asl_recognitio...
86999  space  /content/drive/MyDrive/data_asl/asl_recognitio...

[87000 rows x 2 columns]>

Repartition des donndonnées

In [57]:
x_train,x_holdout = train_test_split(data_df, test_size= 0.10, random_state=42,stratify=data_df[['label']])
x_train,x_test = train_test_split(x_train, test_size= 0.20, random_state=42,stratify=x_train[['label']])

Redimensionnement


In [59]:
img_width, img_height = 64, 64
batch_size = 128
y_col = 'label'
x_col = 'path'
no_of_classes = len(data_df[y_col].unique())


train_datagen = ImageDataGenerator(rescale = 1/255.0, zoom_range=0.2, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.2, horizontal_flip = True, vertical_flip=False)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=x_train,x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height),class_mode='categorical', batch_size=batch_size,
    shuffle=False
)

validation_datagen = ImageDataGenerator(rescale = 1/255.0)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_test, x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height), class_mode='categorical', batch_size=batch_size,
    shuffle=False   
)

holdout_datagen = ImageDataGenerator(rescale = 1/255.0)

holdout_generator = holdout_datagen.flow_from_dataframe(
    dataframe=x_holdout, x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height), class_mode='categorical', batch_size=batch_size,
    shuffle=False
)

Found 62640 validated image filenames belonging to 29 classes.
Found 15660 validated image filenames belonging to 29 classes.
Found 8700 validated image filenames belonging to 29 classes.


In [61]:
base_model = ResNet50V2(input_shape=(64, 64, 3), include_top=False, weights="imagenet")
base_model.trainable = False ## Not trainable weights

94668760/94668760 [==============================] - 1s 0us/step


In [62]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 34, 34, 64)   0           ['conv1_conv[0][0]']             
                                                                                         